In [1]:
!pip install -U -q google-generativeai

In [2]:
#importações
import numpy as np #manipular vetores e cálculos
import pandas as pd #manipulas dataframes
import google.generativeai as genai

GOOGLE_API_KEY = "sua chave de acesso"
genai.configure(api_key=GOOGLE_API_KEY)




In [3]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [5]:
#exemplo:
title = "Perigos do combate criminal com as proprias mãos"
sample_text = ("Titulo:  Perigos do combate criminal com as proprias mãos\n Você pode levar danos e incentivar pessoas.")
embeddings= genai.embed_content(model="models/embedding-001",content=sample_text,title=title,task_type="RETRIEVAL_DOCUMENT")
#argumentos
# title, sample_text_,task_type(tipo de doc)
print(embeddings)

{'embedding': [0.013921131, 0.0019026187, -0.007334547, 0.0054511335, 0.04104718, 0.028382435, -0.044610467, -0.00074191485, 0.0058277817, 0.027623588, 0.0173766, 0.007400321, -0.022996074, 0.0060920073, 0.004667708, 0.0018273259, 0.020477725, 0.0052810265, 0.010640714, -0.010033284, -0.014980455, 0.036039505, -0.045931924, -0.033479463, 0.021996154, -0.028614232, 0.022226075, -0.059282757, -0.036015857, 0.04411108, -0.05978687, 0.03969826, -0.02148368, 0.006627701, 0.027802262, -0.044841036, -0.05459293, 0.045584965, 0.002050605, -0.020645758, -0.029261203, -0.023175377, -0.022948483, -0.0053442214, 0.00038141437, -0.053470887, 0.021722034, -0.008087263, 0.016673785, -0.10304028, 0.011891135, -0.011131515, 0.10252667, 0.0039558196, -0.0316611, -0.06755187, 0.08310152, -0.011061427, -0.04593894, 0.023460906, -0.019656083, 0.009102869, -0.003013073, 0.043534882, -0.016434427, -0.023995189, -0.024734426, -0.015381459, 0.057187457, 0.04179522, -0.009592512, -0.045141246, 0.020352002, 0.00

In [7]:
document1 ={
    "titulo": "Magica para leigos",
    "conteudo": "utilize um baralho para começar"
}
document2={
     "titulo": "Prograamação para leigos",
    "conteudo": "utilize um computador para começar"
}
document3={
     "titulo": "Mecânica para leigos",
    "conteudo": "utilize um carro com defeito para iniciar"
}
document = [document1,document2,document3]


In [8]:
df = pd.DataFrame(document)
df.columns = ["Título","Conteúdo"]
df

,Título,Conteúdo
0,Magica para leigos,utilize um baralho para começar
1,Prograamação para leigos,utilize um computador para começar
2,Mecânica para leigos,utilize um carro com defeito para iniciar


In [9]:
modelo = "models/embedding-001"

In [10]:
def embed_trs(title,text):
  return genai.embed_content(model=modelo,content=text,title=title,task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [13]:
df["embedding"] = df.apply(lambda row: embed_trs(row["Título"],row["Conteúdo"]), axis=1)
df

,Título,Conteúdo,embedding
0,Magica para leigos,utilize um baralho para começar,"[0.037684128, -0.046754543, -0.027372185, -0.0..."
1,Prograamação para leigos,utilize um computador para começar,"[0.01565674, -0.044574566, -0.02646313, 0.0005..."
2,Mecânica para leigos,utilize um carro com defeito para iniciar,"[0.019086415, -0.03681853, -0.033627268, -0.03..."


In [16]:
def gerar_consulta(consulta,base,model):
  embedding_consulta = genai.embed_content(model=modelo,content=consulta,task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["embedding"]),embedding_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteúdo"]



In [17]:
consulta = "como começar uma magica?"
resultado = gerar_consulta(consulta, df, modelo)
print(resultado)

utilize um baralho para começar
